#### Cleaning Players Data

In [20]:
import pandas as pd
import regex as re
from datetime import datetime    
from unidecode import unidecode

In [21]:
players = pd.read_csv("2021.csv", index_col = [0])

In [22]:
players["Year"] = 2021

In [23]:
#Reformatting how club are written
players["Club"] =[unidecode(w) for w in players["Club"]]

players["Player"] =[unidecode(w) for w in players["Player"]]

In [24]:
players["Player"] = [re.sub(' +', ' ', w) for w in players["Player"]]

#### Transforming Values

In [25]:
#Getting the number of days left on contract
Contract_end = []

for w in players["Contract"] : 
    try:
        length = datetime.strptime(str(w), '%b %d, %Y') - datetime(2022,7,1,0,0,0,0)
        Contract_end.append(length.days)  
    except (ValueError, AttributeError): 
        Contract_end.append("NA")

players["Contract"] = Contract_end

In [26]:
#Getting the Age
Age = []

for row in players["Date of Birth"] :
    try : 
        Age.append(re.search(r'\((.*?)\)',row[row.rindex(',')+1:]).group(1))
    except (ValueError, AttributeError): 
        Age.append("NA")
        
players["Date of Birth"] = Age

players = players.rename(columns={'Date of Birth': 'Age'})

In [27]:
#Number of days since they joined the club
Longetivity = []
for w in players["Join"] : 
    try:
        length = datetime(2022,7,1,0,0,0,0) - datetime.strptime(str(w), '%b %d, %Y')
        Longetivity.append(length.days) 
    except (ValueError, AttributeError): 
        Longetivity.append("NA")

players["Join"] = Longetivity

In [28]:
#Converting to numeric
players[['Age', 'Join', 'Contract']] = players[['Age', 'Join', 'Contract']].apply(pd.to_numeric, errors='coerce', axis=1)

In [29]:
#Transforming the market value column
def value_to_float(x):
    x = str(x).replace('€', '')
    if 'k' in x:
        return float(x.replace('k', '')) / 1000
    if 'm' in x:
        return float(x.replace('m', ''))
    return 0

In [30]:
#Formatting
players['Market Value'] = players['Market Value'].apply(value_to_float)

In [31]:
#In case of January transfer/ loan keep the original club
players = players.sort_values('Join', ascending=False).drop_duplicates('Player').sort_index()

#### Dropping Columns

In [32]:
#Contract Column dropped even though I could be kept however contract is new contract
players = players.drop([ "Season", "Club Link", "Contract"], axis=1)

#### Club Cleaning

In [33]:
players["Club"] = [l.replace('FC', '') for l in players["Club"]]

In [34]:
players["Club"] = [l.replace('Wolverhampton Wanderers', 'Wolves') for l in players["Club"]]

In [35]:
players["Club"] = [l.replace('Olympique', '') for l in players["Club"]]

In [36]:
players["Club"] = [l.replace('Hertha BSC', 'Hertha') for l in players["Club"]]

In [37]:
players

,Player Link,Player,Position,Age,Country,Join,Market Value,Club,Year
League,,,,,,,,,
Bundesliga,https://www.transfermarkt.co.uk/konrad-laimer/...,Konrad Laimer,Central Midfield,24.0,Austria,1826.0,26.000,RB Leipzig,2021
Bundesliga,https://www.transfermarkt.co.uk/steven-zuber/p...,Steven Zuber,Left Winger,29.0,Switzerland,696.0,2.500,Eintracht Frankfurt,2021
Bundesliga,https://www.transfermarkt.co.uk/steven-skrzybs...,Steven Skrzybski,Centre-Forward,28.0,Germany,1461.0,1.500,Schalke 04,2021
Bundesliga,https://www.transfermarkt.co.uk/eric-martel/pr...,Eric Martel,Defensive Midfield,19.0,Germany,533.0,1.800,RB Leipzig,2021
Bundesliga,https://www.transfermarkt.co.uk/frederik-soren...,Frederik Sorensen,Centre-Back,29.0,Denmark,2547.0,0.650,1. Koln,2021
...,...,...,...,...,...,...,...,...,...
Süper Lig,https://www.transfermarkt.co.uk/arvydas-noviko...,Arvydas Novikovas,Right Winger,30.0,Lithuania,676.0,0.600,Buyuksehir Belediye Erzurumspor,2021
Süper Lig,https://www.transfermarkt.co.uk/burak-camoglu/...,Burak Camoglu,Right-Back,24.0,Turkey,676.0,0.350,Hatayspor,2021
Süper Lig,https://www.transfermarkt.co.uk/yasin-pehlivan...,Yasin Pehlivan,Defensive Midfield,32.0,Austria,676.0,0.175,Caykur Rizespor,2021


In [38]:
players.to_csv('Players_2021.csv', encoding='utf-8')